# 3. Accounting variables and Ratings
<body> <p  style = "font-family:georgia,garamond,serif;font-size:16px;text-align:justify"></body>


In [1]:
import psycopg2 
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import ast

In [2]:
# connection to the DB
credentials = ast.literal_eval(open(r"C:\Users\pablo\OneDrive - unizar.es\Python Investigación\credentials.txt", "r").read())
conn = psycopg2.connect(dbname="ESGRefinitiv", user=credentials['psycopg2'][0], 
                        password=credentials['psycopg2'][1], host=credentials['psycopg2'][2], port =credentials['psycopg2'][3])
cur = conn.cursor()
engine = create_engine(credentials['sqlalchemy'][0]+ 'ESGRefinitiv')

<body> <p  style = "font-family:georgia,garamond,serif;font-size:22px;text-align:justify"></body>
<b>b11_company_fiscalyear_rates</b></p>
<body> <p  style = "font-family:georgia,garamond,serif;font-size:16px;text-align:justify"></body>
Yearly average exchange rate considering the fiscal year for each company. For each company we obtain the average exchange rate between the end of the fiscal year(t) and the last fiscal period as t less one year.<br>
<b>Tables used:</b><br>
yearly_accounting <br>
classification_iso_currency <br>
exchange_rates <br>
companies_esg_refinitiv <br>

In [3]:
# B11 B11
cur.execute("""
DROP TABLE IF EXISTS 
    b11_company_fiscalyear_rates;
CREATE TABLE
    b11_company_fiscalyear_rates AS
SELECT 
    info.internal_id, EXTRACT(YEAR from cont.yearly_date) as year, cont.end_fiscal_period,  
    (cont.end_fiscal_period - interval '1 year' + interval '1 day') AS  start_fiscal_year, document_currency_iso_id, 
    rate.ds_currency_serie_id, AVG(rate.exchange_rate) AS average_fiscal_exchange_rate 
FROM
    yearly_accounting AS cont 
    INNER JOIN classification_iso_currency AS iso
    ON cont.document_currency_iso_id = iso.currency_iso_id
    INNER JOIN exchange_rates AS rate
    ON iso.ds_currency_serie_usd_id = rate.ds_currency_serie_id
    INNER JOIN companies_esg_refinitiv as info
    ON cont.datastream_id = info.datastream_id
WHERE 
    rate.daily_date_id BETWEEN cont.end_fiscal_period - interval '1 year' + interval '1 day' AND  cont.end_fiscal_period  
GROUP BY
    info.internal_id, EXTRACT(YEAR from cont.yearly_date), cont.end_fiscal_period,  
    cont.end_fiscal_period - interval '1 year',  rate.ds_currency_serie_id,
    document_currency_iso_id;
""")

conn.commit()

<body> <p  style = "font-family:georgia,garamond,serif;font-size:22px;text-align:justify"></body>
<b>b12_accountig_usd</b></p>
<body> <p  style = "font-family:georgia,garamond,serif;font-size:16px;text-align:justify"></body>
Accounting measures in USD based on fiscal year exchanges rates of each company. As explained in the manuscript, we add a column with the year to match the accounting variables with the ESG variables in the next query. <br>
<b> Tables used:</b> <br>
yearly_accounting<br>
companies_esg_refinitiv <br>
b11_company_fiscalyear_rates <br>

In [4]:
cur.execute("""
DROP TABLE IF EXISTS 
    b12_accountig_usd;
CREATE TABLE
    b12_accountig_usd AS
SELECT
    rates.internal_id, rates.year, rates.end_fiscal_period, rates.start_fiscal_year, rates.ds_currency_serie_id, 
    rates.document_currency_iso_id, rates.average_fiscal_exchange_rate, cont.return_on_assets, cont.return_on_equity,
    LOG( NULLIF(cont.number_of_employees, 0) )  AS log_employees, 
    LOG( NULLIF(cont.net_sales * rates.average_fiscal_exchange_rate, 0) ) AS log_net_sales_usd,
    LOG( NULLIF(cont.total_assets * rates.average_fiscal_exchange_rate, 0) ) AS log_total_assets_usd, 
    cont.long_term_debt / NULLIF(cont.total_assets,0) AS long_debt_to_assets,
    cont.total_liabilities /  NULLIF(cont.total_assets,0) AS total_liabilities_to_assets,
    cont.total_liabilities / NULLIF(cont.common_equity,0)  AS total_liabilities_to_equity,
    cont.additions_fixed_assets / NULLIF(cont.total_assets,0) AS additions_fixed_assets_to_assets,
    cont.total_assets, cont.total_liabilities, cont.common_equity, cont.net_sales, cont.net_debt, 
    cont.additions_fixed_assets,
CASE EXTRACT(MONTH from cont.end_fiscal_period)
    WHEN 12 THEN EXTRACT(YEAR from cont.end_fiscal_period)
    ELSE EXTRACT(YEAR from cont.end_fiscal_period) -1
    END year_accounting_toscores
FROM
    yearly_accounting AS cont 
    INNER JOIN companies_esg_refinitiv AS info
    ON info.datastream_id = cont.datastream_id
    INNER JOIN b11_company_fiscalyear_rates AS rates
    ON info.internal_id = rates.internal_id AND EXTRACT(YEAR from cont.yearly_date) = rates.year
WHERE 
    cont.net_sales >=0
""")
conn.commit() 

<body> <p  style = "font-family:georgia,garamond,serif;font-size:22px;text-align:justify"></body>
<b>b31_accountig_isp_ssp</b></p>
<body> <p  style = "font-family:georgia,garamond,serif;font-size:16px;text-align:justify"></body>
We join the table with our accounting variables and the table with CSP, SSP, ISP using the same method as Servaes and Tamayo (2013). Tables used: <br>
b12_accountig_measures_usd : cont <br>
b21_isp_ssp_esg_mv : isp

In [5]:
cur.execute("""
DROP TABLE IF EXISTS 
    b31_accounting_isp_ssp;
CREATE TABLE
    b31_accounting_isp_ssp AS
SELECT 
    b21.internal_id, b12.year AS year_accounting, end_fiscal_period, year_accounting_toscores, b21.year as year_score,
    b21.industry, b21.sector, document_currency_iso_id, b21.ds_country, 
    return_on_assets, return_on_equity, log_employees, log_net_sales_usd, log_total_assets_usd, long_debt_to_assets,
    total_liabilities_to_assets, total_liabilities_to_equity, additions_fixed_assets_to_assets,    
    b21.percent_rank_mv_usd, b21.esg_score_average, b21.environment_pillar_average, b21.social_pillar_average, 
    b21.governance_pillar_average, b21.overall_inclusive, b21.environmental_inclusive, b21.social_inclusive, 
    b21.governance_inclusive, b12.total_assets, b12.total_liabilities, b12.common_equity, b12.net_sales, 
    b12.net_debt, b12.additions_fixed_assets
FROM
    b12_accountig_usd AS b12
    INNER JOIN b21_isp_ssp_esg_mv AS b21 
    ON b21.internal_id = b12.internal_id AND b21.year = b12.year_accounting_toscores
WHERE
    (log_total_assets_usd, log_net_sales_usd, total_liabilities_to_assets, 
    total_liabilities_to_assets, total_liabilities_to_equity, return_on_assets, return_on_equity) IS NOT NULL
    
""")

conn.commit()

<body> <p  style = "font-family:georgia,garamond,serif;font-size:22px;text-align:justify"></body>
<b>Winsorized values</b></p>
<body> <p  style = "font-family:georgia,garamond,serif;font-size:16px;text-align:justify"></body>
We winsorize the variables used in the research 

In [6]:
df_b31 = pd.read_sql('SELECT * FROM b31_accounting_isp_ssp WHERE year_accounting BETWEEN 2010 AND 2019', engine)
variables_winsorized= ['return_on_assets', 'return_on_equity', 'total_liabilities_to_assets', 'total_liabilities_to_equity', 
                      'additions_fixed_assets_to_assets']

for var in variables_winsorized:
    df_b31[[var]] = df_b31[[var]].apply(lambda x: x.clip(x.quantile(0.025,interpolation='nearest'),
                                                  x.quantile(0.975,interpolation='nearest')))

df_b31.to_sql('b31_accounting_isp_ssp_winsorized', engine, if_exists='replace', chunksize=100000, index=False)